# A Simple transformer question and answer model needs:
* Tokinizer (Here i am using BERT's way to tokenize beginning and end of sentences)
* Transformer encoder
* QA head (predict start and end position)
* Training and inference logic



# Use self-built attention head

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math


In [ ]:
config = {
        "vocab_size": 45,
        "hidden_size": 64,
        "max_position_embeddings": 64,
        "num_attention_heads": 4,
        'intermediate_size':10,
        'hidden_dropout_prob':0.01,
        "num_hidden_layers": 12,
        'mask':None
    }

In [ ]:
## Tokinizer

class SimpleTokinizer:
  def __init__(self):
    self.vocab = {"[PAD]":0, "[CLS]":1, "[SEP]":2,"[UNK]":3}
    self.reverse_vocab = {0:"[PAD]", 1:"[CLS]", 2:"[SEP]",3:"[UNK]"}
    self.idx = 4

  def build_vocab(self,texts):
    for text in texts:
      for word in text.lower().split():
        if word not in self.vocab:
          self.vocab[word] = self.idx
          self.reverse_vocab[self.idx] = word
          self.idx+=1
  def encode(self, question, contaxt, max_len = 64):
    ## for each QA, input takes format of [CLS] question tokens [SEP] context tokens [SEP]
    tokens = ["[CLS]"]+question.lower().split()+["[SEP]"]+contaxt.lower().split()+["[SEP]"]
    token_ids = [self.vocab.get(token, self.vocab["[UNK]"]) for token in tokens]
    token_type_ids = [1]*(len(question.split())+2)+[2]*(len(context.split())+1)
    attention_mask = [1] * len(token_ids)
    padding = [0]*(max_len - len(token_ids))
    # print(token_type_ids)
    return {
        'input_ids':torch.tensor(token_ids + padding[:max_len - len(token_ids)]),
        'attention_mask':torch.tensor(attention_mask+padding[:max_len - len(token_ids)]),
        'token':tokens+['[PAD]']*len(padding),
        'token_type_ids':torch.tensor(token_type_ids+padding[:max_len - len(token_ids)]),

    }

In [ ]:
## Sample
question = "What is KNN?"
context = '''KNN, or k-Nearest Neighbors, is a supervised machine learning algorithm used for both classification and regression tasks. It classifies new data points by finding the "k" most similar data points (neighbors) in the training data and assigning the new data point to the majority class among those neighbors.'''


In [ ]:
## Tokenizer
tokenizer = SimpleTokinizer()
tokenizer.build_vocab([question, context])
# input_ids, attention_mask = tokenizer.encode(question, context)
inputs = tokenizer.encode(question, context)

In [ ]:
# inputs

In [ ]:
# input_ids = input_ids.unsqueeze(0)
# attention_mask = attention_mask.unsqueeze(0)

In [ ]:
# config['mask'] = attention_mask

In [ ]:
# input_ids.shape, attention_mask.shape

In [ ]:
## Atttention head

def scaled_dot_product_attention(q,k,v, mask = None):
  # print(q.shape,k.shape,v.shape)
  dim_k = k.size(-1) ## embedding size
  # print(dim_k)
  # print(k.transpose(1,2).shape)
  scores = torch.bmm(q,k.transpose(1,2)) / math.sqrt(dim_k)
  if mask is not None:
    scores = scores.masked_fill(mask==0, -float('inf'))
  weights = F.softmax(scores, dim=1)
  attention_outputs = torch.bmm(weights, v)
  return attention_outputs


class AttentionHead(nn.Module):
  def __init__(self, embed_dim, head_dim, mask=None):
    super().__init__()
    self.q = nn.Linear(embed_dim, head_dim)
    self.k = nn.Linear(embed_dim, head_dim)
    self.v = nn.Linear(embed_dim, head_dim)
    self.mask = mask

  def forward(self,hidden_state):
    attention_outputs = scaled_dot_product_attention(self.q(hidden_state),self.k(hidden_state),self.v(hidden_state), mask = self.mask)

    return attention_outputs


class MultiHeadAttention(nn.Module):
  def __init__(self,config):
    super().__init__()
    embed_dim = config['hidden_size']
    num_heads = config['num_attention_heads']
    head_dim = embed_dim // num_heads
    mask = config['mask']
    self.heads = nn.ModuleList(
        [AttentionHead(embed_dim, head_dim, mask) for _ in range(num_heads)]
    )
    self.output_linear = nn.Linear(embed_dim, embed_dim)

  def forward(self,hidden_state):
    # print(hidden_state.shape)
    # for h in self.heads:
    #   print(h(hidden_state)[0][0].shape)
    # print(self.heads)
    x = torch.cat([h(hidden_state) for h in self.heads], dim = -1)
    x = self.output_linear(x)
    return x



In [ ]:
AttentionHead(config['hidden_size'], config['num_attention_heads'], config['mask'])

In [ ]:
multihead_attn = MultiHeadAttention(config)
token_emb = nn.Embedding(config['vocab_size'], config['hidden_size'])
input_embeds = token_emb(inputs['input_ids'])
input_embeds = input_embeds.unsqueeze(0)
attn_output = multihead_attn(input_embeds)

In [ ]:
class FeedForward(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.linear1 = nn.Linear(config['hidden_size'], config['intermediate_size'])
    self.linear2 = nn.Linear(config['intermediate_size'], config['hidden_size'])
    self.gelu = nn.GELU()
    self.dropout = nn.Dropout(config['hidden_dropout_prob'])

  def forward(self,x):
    x = self.linear1(x)
    x = self.gelu(x)
    x = self.linear2(x)
    x = self.dropout(x)
    return x


In [ ]:
feed_forward = FeedForward(config)

In [ ]:
ff_outputs = feed_forward(attn_output)

In [ ]:
ff_outputs.shape

In [ ]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self,config):
    super().__init__()
    self.layer_norm1 = nn.LayerNorm(config['hidden_size'])
    self.layer_norm2 = nn.LayerNorm(config['hidden_size'])
    self.attention = MultiHeadAttention(config)
    self.feedforward = FeedForward(config)
  def forward(self, x):
    hidden_state = self.layer_norm1(x)
    atten_output =  self.attention(hidden_state)
    x+=atten_output
    x += self.feedforward(self.layer_norm2(x))
    return x


In [ ]:
encoder_layer = TransformerEncoderLayer(config)
print(input_embeds.shape)
encoder_layer(input_embeds).shape

In [ ]:
class Embeddings(nn.Module):
  def __init__(self,config):
    super().__init__()
    self.token_embeddings = nn.Embedding(config['vocab_size'], config['hidden_size'])
    self.position_embeddings = nn.Embedding(config['max_position_embeddings'], config['hidden_size'])
    self.layer_norm = nn.LayerNorm(config['hidden_size'], eps = 1e-12)
    self.dropout = nn.Dropout()

  def forward(self, input_ids):
    seq_length = input_ids.unsqueeze(0).size(1)
    position_ids = torch.arange(seq_length, dtype = torch.long).unsqueeze(0)
    # print(input_ids)
    token_embeddings = self.token_embeddings(input_ids)
    position_embeddings = self.position_embeddings(position_ids)
    embeddings = token_embeddings+position_embeddings
    embeddings = self.layer_norm(embeddings)
    embeddings = self.dropout(embeddings)
    return embeddings



In [ ]:
embedding_layer = Embeddings(config)
embedding_layer(inputs['input_ids'])#.size()

In [ ]:
class TransformerEncoder(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.embeddings = Embeddings(config)
    self.layers = nn.ModuleList(
        [TransformerEncoderLayer(config) for _ in range(config['num_hidden_layers'])]
    )
  def forward(self,x):
    x = self.embeddings(x)
    for layer in self.layers:
      # print(layer)
      x = layer(x)
    return x

In [ ]:
encoder = TransformerEncoder(config)
encoder(inputs['input_ids']).size()

In [ ]:
## Add a QA head
class QA_Transformer(nn.Module):
  def __init__(self, vocab_size, d_model = 64, max_len = 64, heads = 4):
    super().__init__()
    self.config = {
        "vocab_size": vocab_size,
        "hidden_size": d_model,
        "max_position_embeddings": max_len,
        "num_attention_heads": heads,
        'intermediate_size':10,
        'hidden_dropout_prob':0.01,
        "num_hidden_layers": 12,
        'mask':None
    }
    # self.embedding = nn.Embedding(self.config['vocab_size'], self.config['hidden_size'])
    self.encoder = TransformerEncoder(self.config)
    # self.position_embeddings = nn.Parameter(torch.randn(1, self.config['max_position_embeddings'], self.config['hidden_size']))
    # self.position_embeddings = nn.Parameter(torch.randint(1, self.config['max_position_embeddings'], self.config['hidden_size']))
    self.qa_outputs = nn.Linear(self.config['hidden_size'], 2)

  def forward(self, input_ids):#, attention_mask):
    input_ids = input_ids.unsqueeze(0)
    # attention_mask = attention_mask.unsqueeze(0)
    # x = self.embedding(input_ids)+self.position_embeddings[:,:input_ids.size(1)]#.long()
    x = self.encoder(input_ids)#, attention_mask)
    logits = self.qa_outputs(x)
    start_logits, end_logits = logits.split(1,dim=-1)
    return start_logits.squeeze(-1), end_logits.squeeze(-1)




In [ ]:
## Sample
question = "What is KNN?"
context = '''KNN, or k-Nearest Neighbors, is a supervised machine learning algorithm used for both classification and regression tasks. It classifies new data points by finding the "k" most similar data points (neighbors) in the training data and assigning the new data point to the majority class among those neighbors.'''


In [ ]:
## Tokenizer
tokenizer = SimpleTokinizer()
tokenizer.build_vocab([question, context])
# input_ids, attention_mask = tokenizer.encode(question, context)
# input_ids = input_ids.unsqueeze(0)
# attention_mask = attention_mask.unsqueeze(0)
input = tokenizer.encode(question, context)

In [ ]:
# input['input_ids']

In [ ]:
input['attention_mask'].shape, input['input_ids'].shape, len(tokenizer.vocab)

In [ ]:
input['input_ids'].shape

In [ ]:
## Model
model = QA_Transformer(vocab_size =len(tokenizer.vocab), d_model = 64, max_len = 64, heads = 4)
# start_logits, end_logits = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))
start_logits, end_logits = model(input['input_ids'])#, input['attention_mask'])

In [ ]:
# start_logits, end_logits

In [ ]:
# Get answer span
start_idx = torch.argmax(start_logits, dim=1).item()
end_idx = torch.argmax(end_logits, dim=1).item()
print(start_idx, end_idx)
tokens = inputs['input_ids'].tolist()
# print(tokens)
answer = [tokenizer.reverse_vocab.get(t, '[UNK]') for t in tokens[start_idx:end_idx+1]]
print("Predicted answer:", " ".join(answer))

## Model has not been trained yet and no seed is set so output is very unstable

### Output from the above:
46 58
Predicted answer: to the majority class among those neighbors. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]

### But sometimes it could be empty.

# Existing attention head from pytorch

In [ ]:
class SimpleTokenizer:
    def __init__(self):
        self.vocab = {"[PAD]": 0, "[CLS]": 1, "[SEP]": 2, "[UNK]": 3}
        self.reverse_vocab = {0: "[PAD]", 1: "[CLS]", 2: "[SEP]", 3: "[UNK]"}
        self.idx = 4

    def build_vocab(self, texts):
        for text in texts:
            for word in text.lower().split():
                if word not in self.vocab:
                    self.vocab[word] = self.idx
                    self.reverse_vocab[self.idx] = word
                    self.idx += 1

    def encode(self, question, context, max_len=64):
        tokens = ["[CLS]"] + question.lower().split() + ["[SEP]"] + context.lower().split() + ["[SEP]"]
        token_ids = [self.vocab.get(token, self.vocab["[UNK]"]) for token in tokens]
        attention_mask = [1] * len(token_ids)
        padding = [0] * (max_len - len(token_ids))
        return (
            torch.tensor(token_ids + padding[:max_len - len(token_ids)]),
            torch.tensor(attention_mask + padding[:max_len - len(token_ids)])
        )


In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, dim, heads):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim=dim, num_heads=heads, batch_first=True)
        self.norm1 = nn.LayerNorm(dim)
        self.ff = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.ReLU(),
            nn.Linear(dim * 4, dim)
        )
        self.norm2 = nn.LayerNorm(dim)

    def forward(self, x, mask):
        # print(x)
        attn_output, _ = self.attn(x, x, x, key_padding_mask=~mask.bool())
        # print(attn_output)
        x = self.norm1(x + attn_output)
        ff_output = self.ff(x)
        x = self.norm2(x + ff_output)
        return x


In [ ]:
class QA_Transformer(nn.Module):
    def __init__(self, vocab_size, d_model=64, max_len=64, heads=4):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len, d_model))
        self.encoder = TransformerBlock(d_model, heads)
        self.qa_outputs = nn.Linear(d_model, 2)

    def forward(self, input_ids, attention_mask):
        x = self.embedding(input_ids) + self.pos_embedding[:, :input_ids.size(1)]
        # print(x.shape)
        x = self.encoder(x, attention_mask)
        # print(x.shape)
        logits = self.qa_outputs(x)  # [batch, seq_len, 2]
        start_logits, end_logits = logits.split(1, dim=-1)
        return start_logits.squeeze(-1), end_logits.squeeze(-1)


In [ ]:
# Sample data
question = "What is KNN?"
context = '''KNN, or k-Nearest Neighbors, is a supervised machine learning algorithm used for both classification and regression tasks. It classifies new data points by finding the "k" most similar data points (neighbors) in the training data and assigning the new data point to the majority class among those neighbors.'''


# Tokenizer
tokenizer = SimpleTokenizer()
tokenizer.build_vocab([question, context])
input_ids, attention_mask = tokenizer.encode(question, context)

# Model
model = QA_Transformer(vocab_size=len(tokenizer.vocab))
start_logits, end_logits = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))

# Get answer span
start_idx = torch.argmax(start_logits, dim=1).item()
end_idx = torch.argmax(end_logits, dim=1).item()
tokens = input_ids.tolist()
answer = [tokenizer.reverse_vocab.get(t, '[UNK]') for t in tokens[start_idx:end_idx+1]]
print("Predicted answer:", " ".join(answer))


In [ ]:
attention_mask

In [ ]:
tokens[start_idx:end_idx+1]

## Model has not been trained yet and no seed is set so output is very unstable

### output from above:
Predicted answer: is a supervised machine learning algorithm used for both classification and regression tasks. it classifies new data points

### But another run will be different
### Model needs to be trained and set seeds